# ReachOut - reaching movement kinematics analysis
___

## This is the Notebook containing all the steps to prepare the video for analysis with DEEPLABCUT and Anipose and then segment and vizualize the trajectories:

### 0. Cut the video from the FLIR camera into three parts - three different views of the reach
We are recording the videos of the reaching task using the single high speed widefield FLIR camera and multiple mirrors, an then cut the single video into parts corresponding to the Leftside view (CamA), Center view (CamB), Rightside view (CamC). This eliminates the need for synchronisation of different videos, but adds this additional step before we can analyze our videos with DLC. 
As a result of this step you should get three video files with the names -camA, -camB, -camC 

### 1. Analyze the video with DeepLabCut
This step is done using the open-source DEEPLABCUT and Anipose packages, so the tutorial and more info on them can be found on GitHub
__[https://github.com/DeepLabCut](DEEPLABCUT)__

It should be further noted that despite the Anipose documentation advice to use single DLC network for all views we found that better results are achieved with two separate networks trained on the side(1) and front(2) view. That means we start DLC analysis for our videos twice - first time for all the CamA/C videos, and the second time for CamB.
As a result we get two separate sets of _*.csv *.h5_ files. For the next step they should be copied together to the Anipose folder

### 2. Triangulate the 2d trajectories with Anipose 
This step is done using the open-source Anipose package, so the tutorial and more info on them can be found on GitHub
__[https://github.com/lambdaloop/anipose](ANIPOSE)__

To get use of Anipose triangulation we manually put the resulting .csv and .h5 files into anipose project folder _*/Anipose project/pose-2d before_ before starting the triangulation command _anipose triangulate_

As a result of this step you should get the single _*.csv_ file for each session, containing the x,y,z coordinates that we will use further  

### 3. Open the csv file, scroll through the trajectories and manually choose the fragments of the trajectory with the reaches for analysis

As a result of this step you will have the _*.h5_ file with the fragments of the trajectory that you've chosen. Open this file to review the trajectories.  

### 4. Open the h5 file and the corresponding video to review and classify the trajectories

For this script to work point it to the _*.h5_ file you want to analyze (file) and the video corresponding to the same session (file2)
As a result of this step you will have the _*.h5_ file which is used in the vizualisation steps 5. and 6.

### 5. Open the scalars.h5 file and plot all the trajectories of the chosen category

### 6. Open the scalars.h5 file and make violin plots for the chosen parameters and reach categories  
  
  
  ***
  ___










# Analysis part
---

#### 0. Cut the video from the FLIR camera 
Note that we have two scripts for two use cases. If you need to process just a single file, type _from video_split import video_split_ and if you want to process all files in a certain folder type _from video_split_folder import video_split_

In [ ]:
# if you want to do bulk analysis on all the files in a folder - uncomment the second line and comment the first one 
#from video_split import video_split           #for single file
from video_split_folder import video_split    #for multiple files

video_split()

#### 1.  Analyze the video with DeepLabCut
The following line should open DLC interface. Open the project containing your trained network, open the analyze video layout and select the videos you want to track (supposedly, either all camA, camB, or camC videos.

In [ ]:
import deeplabcut
deeplabcut.launch_dlc()

#### 2. Triangulate the 2d trajectories with Anipose 
By now this is done outside of the jupyter notebook, because you need to perform several steps, not including the Anipose calibration: <ol> <li>-  manually make an Anipose directory </li>  <li>-  rename the files after DLC analysis, leaving just the *camX* in the end </li> <li>- transfer the .csv and .h5 files with the tracking results to the Anipose directory </li> <li>- *cd* to this directory and </li> <li>- run from the command prompt *anipose triangulate* </li> </ol>

#### 3. Open the csv file, scroll through the trajectories

The following program snippet allows you to open the csv file with Anipose output and scroll through the traejctories, visualizing them as a 3d plot + 2 projections: to X and Y axis. 
**Scroll** by moving the slider at the very bottom. **The 3d plot can be rotated** by howering on top of it with the mouse and holding left button.
Also when you move your mouse to the left lower X-projection plot and hold the left button you can choose the smaller part of the trajectory to visualize with a **span selector**. 
If after close inspection you find this part of the trajectory useful for further analysis (it looks like a valid reach or any other action you are after), you should press the left **Save button** to add it to the resulting output table.
The **Save_all button** on the right saves the resulting table with all the parameters as a *.h5* file for further analysis.

The parameters calculated and added to the table at this step:
* dX
* dY
* dZ
* dE
* time_difference
* velocity
* acceleration
* jerk

In [ ]:
%matplotlib notebook
# for more dynamic programs like this one we are switching to the interactive matplotlib backend

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
import tracking_split
tracking = tracking_split.TrackingViewer()

#### 4. Open the h5 file and the corresponding video to review and classify the trajectories
The following program opens the file with all the trajectories saved on the previous step, along with the video file **(the path to both should be typed in the following cell)**
Left dropdown list shows all the reaches with the corresponding frames in a video. As soon as you choose one the popup window will show the corresponding part of the video. To **close the window** type Q on the keyboard. After inspecting the video an accompanying plots **mark the reach** with one of the categories on the right. By default all the reaches are marked as Missed or the last chosen option in the **right list**. As soon as you click on one of the options, the label for the current reach gets updated. When you finish **click Save** to write down the changes to the file. The file is saved with the same name as the input file with added *_scalar.h5* suffix and used in all the visualization steps. 

In [ ]:
%matplotlib inline 
# for more static programs

In [ ]:
import viewer
directory = 'C:/Users/daniil.berezhnoi.VAI/Videos/3d plot - manual assembly/NCB1019/'
filename = 'mj_17_18_10_ncb1019.mp4'
filename2 = '12ncb1019.h5'
file = directory+filename
file2 = directory+filename2

view = viewer.ReachesViewer(file, file2)

# Visualization part
___

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
%matplotlib notebook  
# for interactive programs

#### 5. Open the scalars.h5 file and plot all the trajectories of the chosen category
This snippet shows all reaches in the chosen category as Timeseries for visual inspection and analysis. From the lists you can choose the category of reaches and the type of data to plot.

In [ ]:
import reach_view
viewer = reach_view.TimeSeriesViewer()

#### 6. Open the scalars.h5 file and make violin plots for the chosen parameters and reach categories
This snippet shows all the calculated parameters for all reaches in the current session in the form of violin plots. 
The categories to show can be chosen from the left list **(Shift/Ctrl+Click for multiple choise)** and parameters - from the right list.
**Left plot** represents mean for the chosen parameter for every reach (data points).
**Right plot** represents variance for the chosen parameter for every reach (data points).

In [ ]:
import scalar_view
viewer = scalar_view.InteractiveScalarViewer()